<h1>CS152 Assignment 2: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
# Import any packages you need here
# Also define any variables as needed
import copy
from copy import deepcopy
# YOUR CODE HERE (OPTIONAL)

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    # YOUR CODE HERE
    def __init__(self, board, g_cost=0, parent=None): 
        
        self.board = board
        self.parent = parent 
        if self.parent != None:
            self.g_cost = parent.g_cost + 1
        else:
            self.g_cost = 0
        self.h_cost = 0
        self.pruned = False 
        self.tot_cost = self.g_cost + self.h_cost
    
    def __lt__(self,node2):
        
        """
        Compares the total costs of nodes
        """
        
        #compares the total cost of the nodes, will be useful when doing priority queue at the end(Q3)
        return self.tot_cost < node2.tot_cost
    
    def __str__(self):
        """
        Prints out the grid of the board
        """
        
        #initiates empty string to which we are going to add the board entries at any given state
        new_string =''
        
        for i in self.board:
            for j in i:
                new_string += str(j)
                new_string+=' ' 
            new_string+= '\n'
        return new_string
    
    
    def successor_states(self):
        """
        Generates successor states given a board arrangement
        
        """
        
        #we can either move 1 up, down right or left
        possible_dir = ((1,0),(-1,0),(0,1),(0,-1)) 
        
        #list of adjacent tiles to which we can move
        swappable_tiles = []
        
        #finding the position of the empty tile on the board ,adapted from stack overflow https://stackoverflow.com/questions/15886751/python-get-index-from-list-of-lists
        empty_tile = [(i, el.index(0)) for i, el in enumerate(self.board) if 0 in el]
        a = empty_tile[0][0]
        b = empty_tile [0][1]
        #coordinates of our empty tile
        empty_pos = (a,b)
        
        
        for direction in possible_dir:
            
            #using the possible moves to generate coordinates that the blank tile could move to
            new_pos = ((direction[0] + empty_pos[0]), direction[1] + empty_pos[1])
            
            #coordinates of where we will move our empty tile when swapping
            r = new_pos[0]
            s = new_pos[1]
            
            #we make a copy of our board
            duplicate = self.duplicate_board()
            
            #checking that these new places it could move to are still within the board
            if new_pos[1] >=0 and new_pos[1] <= (len(self.board))-1:
                if new_pos[0] >=0 and new_pos[0] <= (len(self.board))-1:
                    if empty_pos[0] >= 0 and new_pos[0] <= (len(self.board))-1 :
            
                        #we swap positions between the blank tile and tiles one move away
                        duplicate.board[a][b],duplicate.board[r][s] = duplicate.board[r][s],duplicate.board[a][b] 
                
                #adds the tiles adjacent to the blank one to the list of swappable tiles
                swappable_tiles.append(duplicate)
                   
        #we return a list of acceptable moves
        return swappable_tiles
    
    
    def duplicate_board(self): 
        
        #we are making a duplicate of a node 
        dupe = copy.deepcopy(self.board) 
        
        # we then set the duplicate as a child of the original board
        return PuzzleNode(dupe, self.g_cost + 1, parent = self) 
    

    #raise NotImplementedError()

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [2]:
# Add any additional code here (e.g. for the memoization extension)

# YOUR CODE HERE (OPTIONAL)

# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    # YOUR CODE HERE
    h = 0
    
    #here , we construct the goal state board so that we can compare it to our current one
    goal_state = copy.deepcopy(state)
    value = 0

    for i in range(len(state)):
        for j in range(len(state)): 
            (goal_state[i][j]) = value
            value += 1
    
    for i in range(len(state)):
        for j in range(len(state)):
            #ensuring that we don't consider 0 in this calculation as it represents the empty tile
            if state[i][j] != 0:
                #comparing current value in the tile to the goal state
                if state[i][j] != goal_state[i][j]: 
                    h += 1 
    return h


# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    # YOUR CODE HERE
    h = 0
    for i in range(len(state)):
        for j in range(len(state)): 
            #ensuring that we don't consider 0 in this calculation as it represents the empty tile
            if state[i][j] != 0:
                #finding the coordinates
                a = state[i][j]%len(state) 
                b = state[i][j]//len(state) 
                
                #we find the absolute difference between current and goal states and add it to the manhattan distance
                dist = abs(a - j) + abs(b - i)
                h+= dist
    return h

    
# Extra heuristic for the extension.  If implemented, modify the function below
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    return 0

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [3]:
# Import any packages or define any helper functions you need here
from queue import PriorityQueue 
# YOUR CODE HERE (OPTIONAL)
# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    # YOUR CODE HERE
    steps = 0
    exp = 0
    max_frontier = 0
    opt_path = 0
    err = 0
    frontier_size = 0
    
    #this is the priority queue where we will have our frontier
    frontier = PriorityQueue()
    
    #this is the state in which we are starting from
    initial_state = PuzzleNode(state)
    
    #these are the nodes we have tackled 
    visited_states = {} 
    
    #computing the goal arrangement for our given state
    goal = copy.deepcopy(state)
    value = 0
    for i in range(len(state)):
        for j in range(len(state)): 
            goal[i][j] = value 
            value += 1
    
    #checking to ensure that no values are repeated
    values_list = []
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] not in values_list:
                values_list.append(state[i][j])
    if len(values_list) != (len(state))**2:
        print("error:some values are repeated")
        err = -1
    
    
    #checking that it is an n*n board by ensuring that no. of rows and columns are equal
    if len(state) != len(state[0]):
        print("error:board not a square matrix")
        err = -1 
    
    #checking to ensure that all the rows stretch to the full length   
    for i in state:
        if len(i) != len(state):
            print("error:rows not of the same length")
            err = -1
    
    #since we are at the root node, the g_cost = 0
    initial_state.g_cost = 0
    initial_state.tot_cost = heuristic(initial_state.board)
    
    #we add the initial state to the frontier
    frontier.put(initial_state)
    
    #increase frontier size by 1 since we have added a node to it
    frontier_size += 1
    
    #we add. it to the visited states
    visited_states[str(initial_state.board)] = initial_state 
    
    #we repeat the processes below as long as we have a node in the frontier and have not reached the goal state
    while frontier.qsize() > 0:
        #updating maximum frontier size
        max_frontier = max(frontier_size,max_frontier)
        
        #getting value with least cost from the frontier, at first, this will just be our initial state
        current = frontier.get()
        #reducing frontier size by 1 since we have taken off a value
        frontier_size -= 1
        #if a node has already been pruned, then we don't address it
        if current.pruned: 
            continue
        
        #checks if we have arrived at the desired arrangement
        if current.board == goal:
            print("You have solved the puzzle!")
            break
        
        #generating successors of the current state
        successors = current.successor_states()
        #increasing no. of expanded states by 1 since we have expanded a node to get its descendants
        exp += 1
        
        
        for node in successors:
            #finding the heuristic value of each successor
            node.h_cost = heuristic(node.board)
            #adding heuristic value of each successor to their g_cost
            node.tot_cost = node.g_cost + node.h_cost 
            
            #adding nodes that have not been tackled to the frontier for consideration
            if (str(node.board)) not in visited_states: 
                frontier.put(node)
                
                #updating size of frontier after adding the unvisited nodes
                frontier_size += 1 
                
                #adding these nodes to the visited states since we have already put them in the frontier for consideration
                visited_states[(str(node.board))] = node
            
            else:
                #comparing the parent's g_cost to that of its successors
                if visited_states[str(node.board)].g_cost > current.g_cost + 1:
                    
                    #removing those whose cost is too high to be considered for next node
    
                    visited_states[str(node.board)].pruned = True 
    
    #initiating the optimal path list
    goal_path = []
    goal_path.append(current.board)
    #adding each node that was expanded to get to the goal state to the goal_path
    while current.parent:
        current = current.parent 
        goal_path.append(current.board)  
    steps = len(goal_path)-1
    
    #reversing the path so as to obtain an intial--->final state solution 
    opt_path = []
    for node in goal_path:
        opt_path = [node] + opt_path
    
    
    return steps,exp,max_frontier,opt_path, err
    
    


<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [4]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

error:some values are repeated


In [5]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [6]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


You have solved the puzzle!
You have solved the puzzle!
You have solved the puzzle!
You have solved the puzzle!
You have solved the puzzle!
You have solved the puzzle!


In [7]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

You have solved the puzzle!
You have solved the puzzle!


## References

Belwariar, R. (2018, September 7). A* Search Algorithm - GeeksforGeeks. GeeksforGeeks. https://www.geeksforgeeks.org/a-search-algorithm/

Rohan, S. (2022, September 20). Template A*Search. Minerva Forum. https://sle-collaboration.minervaproject.com/?minervaNotebookId=cl7xiiajg010j0j2g4f6573u9&userId=10905&name=Aroma+Atieno&avatar=https%3A//s3.amazonaws.com/picasso.fixtures/Aroma_Atieno_10905_2020-09-01T20%3A03%3A26.357Z&noPresence=1&readOnly=0&isInstructor=0&signature=3c1ed9e7eb6bb262b81b37054ba9808363894cc62f213ecf343694d4dc7b85b2

Stack Overflow. (2013, June). Python, get index from list of lists. Stack Overflow. https://stackoverflow.com/questions/15886751/python-get-index-from-list-of-lists

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [8]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

AssertionError: 

In [ ]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert(dom > 0)

In [ ]:
## Memoization test - will be carried out after submission